In [4]:
import torch
import gc
from transformers import AutoTokenizer, LlamaForCausalLM


gc.collect()
torch.cuda.empty_cache()
model_directory = "/home/ksaff/Desktop/ttyd/query_llama"
tokenizer = AutoTokenizer.from_pretrained(model_directory)
model = LlamaForCausalLM.from_pretrained(model_directory,
                                         load_in_8bit=True,
                                         device_map={'': 0})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.20it/s]


In [207]:
import os

with open(f"{os.environ.get('SQL_FILES_PATH')}/ddl.sql", "r") as file:
    ddl = file.read()

input = ddl
prompt = 'Make SQLite query based on DDL and instruction.'
instruction = 'Fetch me names of 5 poeple whos phone number starts with 6'
text = (
    prompt
    + '### Instruction:\n'
    + instruction
    + '### Input:\n'
    + input
    + '### Response:\n'
)

In [208]:
input_ids = tokenizer.encode(text, return_tensors='pt')
input_ids = input_ids.to('cuda')

In [209]:
output = model.generate(input_ids, max_length=1000, temperature=0.3, num_return_sequences=1)

In [210]:
import re

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

match = re.search(r'### Response:\s*(.*)', generated_text)
response = match.group(1)
response

"SELECT * FROM JPK_ENTITY WHERE PHONE LIKE '%6%';"

In [211]:
import sqlite3
from dotenv import load_dotenv
load_dotenv()

conn = sqlite3.connect(os.environ.get('SQLITE_PATH'))
c = conn.cursor()

def execute_query(query):

    c.execute(query)
    result = c.fetchall()

    # Extract column names
    columns = [desc[0] for desc in c.description]
    
    # Prepare the response
    response = {"query": query, "headers": columns, "data": result}

    return response

In [212]:
result = execute_query(response)
print(f'User instruction: {instruction}')
print(f'model output: {response}')
'Result query:', result['data']

User instruction: Fetch me names of 5 poeple whos phone number starts with 6
model output: SELECT * FROM JPK_ENTITY WHERE PHONE LIKE '%6%';


('Result query:',
 [(788356,
   566911,
   '432456233480',
   'Vincenzo',
   'Ballesterous',
   19521107,
   528919762),
  (522471, 778771, '890714464769', 'Ping', 'Cashin', 20010131, 89398966),
  (670648,
   393345,
   '089186526308',
   'Delmar',
   'Motteshard',
   19520306,
   978416963),
  (210860, 707961, '490314661706', 'Floy', 'Makhija', 20040905, 5776770),
  (726388, 132911, '776040489615', 'Misty', 'Paolucci', 19950525, 688991892),
  (6877, 228540, '615087090673', 'Aaron', 'Lenigan', 19591007, 966587907),
  (732641, 720300, '696056077660', 'Janita', 'Gillerist', 19781106, 1664993),
  (41032, 632999, '928819963883', 'Coleman', 'Pritt', 19821005, 54355636),
  (90609, 343063, '010326169960', 'Mitsuko', 'Fukuroku', 19720509, 265196963),
  (916890, 208362, '729184696197', 'Rico', 'Yopp', 19630706, 256663751),
  (561468,
   163310,
   '267839876410',
   'Annemarie',
   'Bernatchez',
   19731221,
   823863246),
  (541352, 90558, '463015288622', 'Lawanda', 'Mclilly', 19841126, 677115